In [1]:
from feature.clip_make_dataset import make_data
from feature.after_dataset import accumulate
from feature.utils import log_scale
from model.model_train import model_train
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

c:\Python\Python38\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [3]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')

In [4]:
# log_scale(train, test)

In [5]:
train = train.reset_index(inplace=False)
test  = test.reset_index(inplace=False)
train = pd.concat([train, p_train], axis=1)
test = pd.concat([test, p_test], axis=1)
train = train.drop(['index'], axis=1)
test = test.drop(['index'], axis=1)

In [6]:
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, task_type="GPU", random_seed=113)
model_train(train, test, cat)

Learning rate set to 0.037333
0:	learn: 40.7248814	total: 8.18ms	remaining: 8.17s
100:	learn: 8.5618510	total: 861ms	remaining: 7.66s
200:	learn: 6.3184465	total: 1.7s	remaining: 6.77s
300:	learn: 5.5031912	total: 2.54s	remaining: 5.91s
400:	learn: 5.1498683	total: 3.37s	remaining: 5.03s
500:	learn: 5.0320777	total: 4.2s	remaining: 4.18s
600:	learn: 4.8358090	total: 4.99s	remaining: 3.31s
700:	learn: 4.3850100	total: 5.83s	remaining: 2.48s
800:	learn: 4.1687336	total: 6.61s	remaining: 1.64s
900:	learn: 4.0007056	total: 7.41s	remaining: 814ms
999:	learn: 3.7640334	total: 8.2s	remaining: 0us
validation rmse: 8.872315882306655
Learning rate set to 0.039126
0:	learn: 40.3117475	total: 9.58ms	remaining: 9.57s
100:	learn: 7.7108864	total: 825ms	remaining: 7.35s
200:	learn: 5.6924829	total: 1.63s	remaining: 6.47s
300:	learn: 5.0305454	total: 2.42s	remaining: 5.62s
400:	learn: 4.5752716	total: 3.16s	remaining: 4.72s
500:	learn: 4.2304475	total: 3.86s	remaining: 3.84s
600:	learn: 3.9920866	tota

In [7]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp.to_csv('feature_imp.csv', encoding='cp949')

In [8]:
train.columns

Index(['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
       '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
       '일간누적청색광량', '시간당총광량', '일간누적총광량', 'predicted_weight_g', 'Case', '6time',
       'diff_temp', 'dark', 'water', '월간누적분무량', '월간누적백색광량', '월간누적청색광량',
       '월간누적적색광량', '('일간누적백색광량', 1.0)', '('일간누적백색광량', 2.0)',
       '('일간누적백색광량', 3.0)', '('일간누적백색광량', 4.0)', '('일간누적분무량', 1.0)',
       '('일간누적분무량', 2.0)', '('일간누적분무량', 3.0)', '('일간누적분무량', 4.0)',
       '('일간누적적색광량', 1.0)', '('일간누적적색광량', 2.0)', '('일간누적적색광량', 3.0)',
       '('일간누적적색광량', 4.0)', '('일간누적청색광량', 1.0)', '('일간누적청색광량', 2.0)',
       '('일간누적청색광량', 3.0)', '('일간누적청색광량', 4.0)', '('일간누적총광량', 1.0)',
       '('일간누적총광량', 2.0)', '('일간누적총광량', 3.0)', '('일간누적총광량', 4.0)'],
      dtype='object')